In [ ]:
# Connect to Postgresql 

import psycopg2

conn = psycopg2.connect(
    dbname="database_name",
    user="username",
    password="password",
    host="host",
    port="port"
)

cur = conn.cursor()
cur.execute("SELECT * FROM Table")

rows = cur.fetchall()
for row in rows:
    print(row)

cur.close()
conn.close()

In [ ]:
# Logging settings
import logging
from logging.handlers import RotatingFileHandler
import os 
from os.path import join
import datetime
from datetime import datetime 


log = "logs"
os.makedirs(log, exist_ok=True)

dt_st = datetime.now()
log_path = join(log,f"debug_{dt_st.strftime('%Y%m%d')}.log")    
log_formatter = logging.Formatter('%(asctime)s [%(levelname)s]: %(message)s')
log_handler = RotatingFileHandler(filename=log_path, maxBytes=1048576, backupCount=10, delay=True)
log_handler.setFormatter(log_formatter)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)  
logger.addHandler(log_handler)

In [ ]:
#  Create Table 

import ibm_db


try:
    db2_connection_string = "DATABASE=AYB_APPL;HOSTNAME=10.143.16.244;PORT=50000;PROTOCOL=TCPIP;UID=IOT_DATA;PWD=asd23fgh;"

    conn = ibm_db.connect(db2_connection_string, "", "")

    # Create a table in the IOT_Data schema
    sql = '''
        CREATE TABLE IOT_Data.threshold_alert (
            CATEGORY VARCHAR(20) NOT NULL,
            EQP VARCHAR(20) NOT NULL,
            TS_DT TIMESTAMP NOT NULL,
            DATA_COLUMN VARCHAR(20) NOT NULL,
            STATE DECIMAL(1) NOT NULL,
            DATA_VALUE DECIMAL(15, 6) NOT NULL,
            THRESHOLD_TYPE DECIMAL(1),
            THRESHOLD_VALUE DECIMAL(15, 6),
            INPUT_DT TIMESTAMP,
            INPUT_PERSON VARCHAR(20),
            INPUT_COMMENT VARCHAR(1000),
            PRIMARY KEY (CATEGORY, EQP, TS_DT, DATA_COLUMN)
        )
    '''
    stmt = ibm_db.exec_immediate(conn, sql)
    ibm_db.commit(conn)

except Exception as e:
    print("Error:", e)

finally:
    if conn:
        ibm_db.close(conn)

In [ ]:
# INSERT (Databasega bitta butun qilib insert qilish)
conn_str = "DATABASE=AYB_APPL;HOSTNAME=10.143.16.244;PORT=50000;PROTOCOL=TCPIP;UID=IOT_DATA;PWD=asd23fgh;"
conn = ibm_db.connect(conn_str, "", "")

columns = ', '.join(df.columns)

data = tuple(tuple(row) for row in df.values)
values = ",".join(map(str, data))

insert_query = f"INSERT INTO THRESHOLD_ALERT ({columns}) VALUES {values}"

stmt = ibm_db.prepare(conn, insert_query)

try:
    if ibm_db.execute(stmt):
        print("Inserted to Database")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# INSERT(Bu usul sal qiyinroq :NG)
def insert_to_database(df):   
    try:
        columns = ['Column_1', 'Column_2', 'Column_3', 'Column_4']
        rows = [tuple(row) for row in df[columns].values]
        values = ",".join(["%s"] * len(columns))

        sql_query = f"INSERT INTO Table_Name ({', '.join(columns)}) VALUES ({values})"

        conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)

        try:
            with conn.cursor() as cursor:
                cursor.executemany(sql_query , rows)
                conn.commit()
                logging.info(f"Data inserted into Table_Name")
                print("Data inserted into Table_Name")
                
        except Exception as e:
            logging.info(e)
            print(e)
        
    except Exception as e:
        print("Error while connecting to PostgreSQL:", e)

import ibm_db

def insert_to_database(df):
    try:
        columns = df.columns
        rows = tuple(tuple(row) for row in df.values)
        values = ",".join(map(str, rows))
        conn_str = "DATABASE=*****;HOSTNAME=*******;PORT=*******;PROTOCOL=******;UID=*****;PWD=******;"
        conn = ibm_db.connect(conn_str, "", "")
        insert_query = f"INSERT INTO threshold_alert ({', '.join(columns)}) VALUES ({values})"
        stmt = ibm_db.prepare(conn, insert_query)
    
        try:
            if ibm_db.execute(stmt, values):
                print(f"Inserted to Database")   
        except Exception as e:
            print("Error while connecting to IBM Db2:", e)

    except Exception as e:
        print("Error while connecting to IBM Db2:", e)

insert_to_database(df)

In [ ]:
# Insert qilish uchun funksiya : OK

def insert_query(df):
    values = ''
    for index, row in df.iterrows():
        value_1 = row['column_1']
        value_2 = row['column_2']
        value_3 = row['column_3']
        value_4 = row['column_4']
        value_5 = row['column_5']
        value_6 = f"{row['column_6']}, {row['column_7']}, {row['column_8']}, {row['column_9'], {row['column_10']}}"
        
        values += f"('{value_1}','{value_2}', '{value_3}', '{value_4}', '{value_5}', '{value_6}'),"   
    values = values[:-1] 

    query = f"INSERT INTO IOT_DATA.THRESHOLD_ALERT(COLUMN_1, COLUMN_2, COLUMN_3, COLUMN_4, COLUMN_5, COLUMN_6) VALUES {values};"
    return query

insert_query(df)

In [4]:
# Folder ichidegi fayllarni bitta DF ga yuklab olish :OK
import glob
import pandas as pd 

def read_and_concat_csv_files_from_folder(folder_path):
    files = glob.glob(folder_path +"\*.csv")    
    dfs = []
    for file in files:
        i = pd.read_csv(file, encoding="cp932")
        dfs.append(i)
    df = pd.concat(dfs, axis=0, ignore_index = True)
    return df


# Example usage:
folder_path = r"C:\Desktop\Folder"
result_df = read_and_concat_csv_files_from_folder(folder_path)

In [ ]:
# Folder ichidegi Sub folderlar fayllarni bitta DF ga yuklab olish :OK
import os 
import pandas as pd

def read_and_concat_csv_files_from_subfolder(path):
    df = pd.DataFrame()
    for root, dirs, files in os.walk(path):
        for i in files:
            if i.endswith(".csv"):
                full = os.path.join(root, i)
                df = pd.read_csv(full, encoding="cp932")
                dfs = pd.concat([dfs, df], axis=0, ignore_index=True)
    return dfs

# Example usage:
path = r"C:\Users\00220401626\Desktop\General.csv"
result_df = read_and_concat_csv_files_from_subfolder(path)

In [ ]:
# Teamsga  message jo'natishni {oddiy} usuli  :OK
import requests
import json

def send_teams_message(webhook_url, message):
    headers = {"Content-Type": "application/json"}
    payload = {"text": message}

    response = requests.post(webhook_url, data=json.dumps(payload), headers=headers, verify=False)

    if response.status_code == 200:
        print("Message sent successfully to Microsoft Teams!")
    else:
        print(f"Failed to send message. Status code: {response.status_code}, Response: {response.text}")

teams_webhook_url = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1"
message_to_send = "This is a 3 test message!"
send_teams_message(teams_webhook_url, message_to_send)

In [ ]:
# Teamsga bitta qilib messagelarni jo'natish :OK
import requests 
import json

def send_teams_message(df,teams_webhook_url):

    message_text = ""
    for index, row in df.iterrows():
        TITLE = row['Column_1']
        value_1 = row['Column_2']  
        value_2 = row['Column_3']  
        value_3 = row['Column_4']  
        value_4 = row['Column_5'] 
        value_5 = row['Column_6']  
        value_6 = row['Column_7'] 

        message_text += f"{TITLE}\n\n判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\n対象データ名: {value_4}\n\nデータ値: {value_5}\n\nしきい値: {value_6}\n\n↓MotionBoardへのリンク\n\n----------------------------\n\n"
    
    payload = {"text": message_text}
    response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)

    if response.status_code == 200:
        print("Message sent successfully to Microsoft Teams!")
    else:
        print(f"Failed to send message. Status code: {response.status_code}, Response: {response.text}")

# Example usage:
send_teams_message(df, teams_webhook_url)

In [ ]:
# Teamsga for loop orqali messagelarni jo'natish :OK

def send_teams_message(df):
    teams_webhook_url = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1"

    for index, row in df.iterrows():
        value_0 = row['Column_1']
        value_1 = row['Column_2']  
        value_2 = row['Column_3']  
        value_3 = row['Column_4']  
        value_4 = row['Column_5'] 
        value_5 = row['Column_6']  
        value_6 = row['Column_7'] 
        
        message = f"{value_0}\n\n判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\n対象データ名: {value_4}\n\nデータ値: {value_5}\n\nしきい値: {value_6}\n\n↓MotionBoardへのリンク\n\n"

        payload = {"text": message}

        response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)

        if response.status_code == 200:
            print("Message sent successfully to Microsoft Teams!")
        else:
            print(f"Failed to send message. Status code: {response.status_code}, Response: {response.text}")
            
send_teams_message(df)

# if index % 5 == 0:  # Send message every 5 iterations
#     payload = {"text": message}
#     response = requests.post(teams_webhook_url, data=json.dumps(payload), verify=False)

In [ ]:
# JSON dan ma'lumotlarni olish :OK 
import datetime as dt
import json

try:    
    with open(json_path) as json_file:
        config = json.load(json_file)

    # Extract Configuration values
    database = config['database']
    host = config['host']
    port = config['port']
    protocol = config['protocol']
    user = config['user']
    password = config['password']
    log = config['log']  
    insertion_time = str(config['insertion_time'])
    days = int(config['days'])
    inserted_time = dt.datetime.strptime(insertion_time, '%Y-%m-%d %H:%M:%S')
    inserted_day = inserted_time.strftime('%Y%m%d')

except Exception as e:
    logging.error(f"An error occurred while loading configuration: {e}")

In [ ]:
# JSON ni update qilish  :OK
def update_json(df, json_path):
    config['insertion_time'] = str(df['Timestamp'].max())
    json_data = json.dumps(config, indent=2)

    with open(json_path, 'w') as output_file:
        output_file.write(json_data)

update_json(df, json_path)

In [12]:
# join method stringlarni bog'lash uchun ishlatiladi :OK

my_list = ["apple", "banana", "orange"]
result = ",".join(my_list)
print(result)

apple,banana,orange


In [ ]:
# DELETE Ibm_db databasedan tableni o'chirib tashlash  :OK

delete_query = "DELETE FROM THRESHOLD_ALERT"
try:
    ibm_db.autocommit(conn, ibm_db.SQL_AUTOCOMMIT_OFF)
    if ibm_db.exec_immediate(conn, delete_query):
        print("Existing data deleted successfully")

    ibm_db.commit(conn)

except Exception as e:
    print("Error:", e)
    ibm_db.rollback(conn)

In [ ]:
# Database Connection Test  :OK
import psycopg2
from psycopg2 import OperationalError
import logging
import time

database = "fms01"
host = "10.143.64.54"
port = "5432"
protocol = "TCPIP"
user = "postgres"
password = "fms"

def test_connection(database, user, password, host, port):
    try:
        conn = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)
        conn.close()
        logging.info("Connection to database successful")
    except OperationalError as e:
        logging.info(f"Unable to connect to the database: {e}")
        time.sleep(10)

test_connection(database, user, password, host, port)

In [ ]:
data = tuple(tuple(row) for row in df.values)
values = ",".join(map(str, data))
columns = ', '.join(df.columns)
placeholders = ",".join(["?" for _ in columns])

In [ ]:
import ibm_db

def insert_to_database(df):
    try:
        columns = df.columns
        conn_str = "DATABASE=AYB_APPL;HOSTNAME=10.143.16.244;PORT=50000;PROTOCOL=TCPIP;UID=IOT_DATA;PWD=asd23fgh;"
        conn = ibm_db.connect(conn_str, "", "")
        
        placeholders = ",".join(["?" for _ in columns])
        insert_query = f"INSERT INTO threshold_alert ({', '.join(columns)}) VALUES ({placeholders})"
        stmt = ibm_db.prepare(conn, insert_query)
    
        try:
            values_list = [tuple(row) for row in df.values]
            if ibm_db.execute_many(stmt, values_list):
                print("Inserted to Database")
        except Exception as e:
            print("Error during insert:", e)
        
        ibm_db.close(conn)

    except Exception as e:
        print("Error while connecting to IBM Db2:", e)

# Example usage:
insert_to_database(df)


In [ ]:
data = tuple(tuple(row) for row in df.values)
values = ",".join(map(str, data))
columns = ', '.join(df.columns)
placeholders = ",".join(["?" for _ in columns])


conn_str = "DATABASE=AYB_APPL;HOSTNAME=10.143.16.244;PORT=50000;PROTOCOL=TCPIP;UID=IOT_DATA;PWD=asd23fgh;"

columns = ', '.join(df.columns)
data = tuple(tuple(row) for row in df.values)
placeholders = ', '.join(['(' + ', '.join(['?' for _ in row]) + ')' for row in data])

insert_query = f"INSERT INTO threshold_alert ({columns}) VALUES {placeholders}"

stmt = ibm_db.prepare(conn_str, insert_query)

try:
    if ibm_db.execute(stmt, data):
        print("Inserted to Database")
except Exception as e:
    print(f"Error: {e}")
    print("Failed to insert into the database")

sql_query = f"INSERT INTO threshold_alert ({columns}) VALUES {values}"
insert_query = f"INSERT INTO threshold_alert ({', '.join(columns)}) VALUES ({placeholders})"

data = [tuple(row) for row in df.values]
values = ",".join(["(" + ",".join(map(str, row)) + ")" for row in data])
columns = ', '.join(df.columns)
placeholders = ",".join(["?" for _ in columns])

In [ ]:
# 3
def send_data_to_teams(df):
    teams_url = "https://kyoceragp.webhook.office.com/webhookb2/32c8e9d8-fee1-47b5-8f88-7c024b169401@82cc187e-25d5-45e4-8c34-8434bf6075fe/IncomingWebhook/c008d92acff64b93a35fa912edecdbf3/329f5970-aad5-4b39-90b5-cf6569c670e1"             

    myTeamsMessage = pymsteams.connectorcard(teams_url)
    
    for index, row in df.iterrows():
        TITLE = row['Column_1']
        value_1 = row['Column_2']  
        value_2 = row['Column_3']  
        value_3 = row['Column_4']  
        value_4 = row['Column_5'] 
        value_5 = row['Column_6']  
        value_6 = row['Column_7'] 
        link = "http://mb.in.kyocera.co.jp/motionboard/main?mbid=fid5uk"

        myTeamsMessage.title(f"{TITLE}")
        myTeamsMessage.text(f"判定: {value_1}\n\n種別: {value_2}\n\n装置: {value_3}\n\n対象データ名: {value_4}\n\nデータ値: {value_5}\n\nしきい値: {value_6}\n\n↓MotionBoardへのリンク\n\n{link}")
        myTeamsMessage.send()
        time.sleep(3)